In [1]:
import os
import time
import numpy as np
import soundfile as sf
import scipy
import csv
import scipy.misc
import imageio
import shutil
import sys
import datetime
import random
import collections
import pickle
import math

from random import randint

from keras.backend.tensorflow_backend import set_session
import tensorflow as tf
# set the memory usage
tf_config = tf.ConfigProto()
tf_config.gpu_options.allow_growth = True
set_session(tf.Session(config=tf_config))

from keras.layers import MaxPooling1D, Dropout, AveragePooling2D,LSTM,Reshape, Bidirectional, Add
from keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D, Input, BatchNormalization,Conv1D
from keras.models import Model, load_model
from keras.callbacks import EarlyStopping
from keras import backend as K
from keras import losses, optimizers
from keras.layers.pooling import GlobalMaxPooling1D

Using TensorFlow backend.


In [2]:
frames_per_sample = 200

def create_melspec_train(path):
    train_frames=[]
    train_labels=[]
    with open('event_list_devtrain_glassbreak.csv', 'rt') as train_file:
        reader = list(csv.reader(train_file, delimiter='\n'))
        
        for row,i in zip(reader,range(5000)):
            fields= str(row).split('\\t')
            
            temp_train_frames =[]
            temp_train_labels = []
                    
            if(len(fields)>1):
                if(fields[1] and fields[2]):
                    file_name = fields[0].replace("[\'", "").replace("\']", "")
                    melspec = imageio.imread(path + str(i) + '.png')
                    print(file_name)
                    event_start_index = int(np.round(float(fields[1])/30 * 1305))
                    event_finish_index = int(np.round(float(fields[2])/30 * 1305))
                    event_length = event_finish_index - event_start_index 
                    print(event_start_index)
                    print(event_finish_index)
                    
                    #calculating sample start and finish indices, randomly
                    sample_start = randint(max(0,event_start_index - (200-event_length)),event_start_index)
                    sample_finish = min(1305,sample_start+frames_per_sample)
                    
                    print('before')
                    print(sample_start)
                    print(sample_finish)
                    sample_length = sample_finish - sample_start 
                    if sample_length < frames_per_sample:#if we don't have enough negative samples after event
                        sample_start = sample_start - (frames_per_sample - sample_length)
                    
                    print('after')
                    print(sample_start)
                    print(sample_finish)
                    
                    temp1 = np.array(melspec)
                    temp_train_frames = temp1[:,sample_start:sample_finish]
                    
                    print(np.array(temp_train_frames).shape)
                    for ind in range(sample_start,event_start_index):
                        temp_train_labels.append(0)
                    for ind in range(event_start_index,event_finish_index):
                        temp_train_labels.append(1)
                    for ind in range(event_finish_index,sample_finish):
                        temp_train_labels.append(0)
                    
                    print('temp_train_labels.shape is: '+str(np.array(temp_train_labels).shape))   
                    temp2 = np.array(temp_train_frames)
                    temp2 = temp2.reshape(128,200)
                    train_frames.append(temp2)
                    
                    temp3 = np.array(temp_train_labels)
                    temp3 = temp3.reshape(200)
                    train_labels.append(temp3)
                    
                    print(np.array(train_frames).shape)
                    print(np.array(train_labels).shape)
            if(i%500==0):
                print('Reading files: '+str(i))
    
    train_file.close()
    train_data = {'train_frames':train_frames,
                 'train_labels':train_labels}
    return train_data

In [3]:
file_path = 'Datasets_legMel_44100_128/train_5000_glassbreak_Prob1_44100_128/'
d={}
d = create_melspec_train(file_path)

mixture_devtrain_glassbreak_000_bc7f7690c229e807c86b04290ac3944c.wav
917
954
before
798
998
after
798
998
(128, 200)
temp_train_labels.shape is: (200,)
(1, 128, 200)
(1, 200)
Reading files: 0
mixture_devtrain_glassbreak_001_34faf0663d92693e04c9737ce2985718.wav
216
270
before
94
294
after
94
294
(128, 200)
temp_train_labels.shape is: (200,)
(2, 128, 200)
(2, 200)
mixture_devtrain_glassbreak_002_8151cfa7030ceef3f1dbe44a1d6533f0.wav
1173
1206
before
1125
1305
after
1105
1305
(128, 200)
temp_train_labels.shape is: (200,)
(3, 128, 200)
(3, 200)
mixture_devtrain_glassbreak_003_0447bfc9e95f02a2283a6fe9542388a4.wav
733
777
before
711
911
after
711
911
(128, 200)
temp_train_labels.shape is: (200,)
(4, 128, 200)
(4, 200)
mixture_devtrain_glassbreak_004_37093e4dfc76ee6c9443f2e958408cf1.wav
744
784
before
590
790
after
590
790
(128, 200)
temp_train_labels.shape is: (200,)
(5, 128, 200)
(5, 200)
mixture_devtrain_glassbreak_005_811fd0e98ff6d37bd1fd91741754564b.wav
41
89
before
21
221
after
21
221
(1

after
379
579
(128, 200)
temp_train_labels.shape is: (200,)
(47, 128, 200)
(47, 200)
mixture_devtrain_glassbreak_047_3eb1be84387ece26a0ef1b20ae1e65c2.wav
1189
1258
before
1123
1305
after
1105
1305
(128, 200)
temp_train_labels.shape is: (200,)
(48, 128, 200)
(48, 200)
mixture_devtrain_glassbreak_048_329715c4a85d0a58ec0d8abf7f7fb1f5.wav
971
1040
before
915
1115
after
915
1115
(128, 200)
temp_train_labels.shape is: (200,)
(49, 128, 200)
(49, 200)
mixture_devtrain_glassbreak_049_1c24a7844910c4769039c066c800a91f.wav
668
754
before
604
804
after
604
804
(128, 200)
temp_train_labels.shape is: (200,)
(50, 128, 200)
(50, 200)
mixture_devtrain_glassbreak_050_7674ebb88883a29013571467bfcfe34f.wav
393
425
before
286
486
after
286
486
(128, 200)
temp_train_labels.shape is: (200,)
(51, 128, 200)
(51, 200)
mixture_devtrain_glassbreak_051_ff19eb58e296a8019ed885f3791fca3f.wav
553
576
before
452
652
after
452
652
(128, 200)
temp_train_labels.shape is: (200,)
(52, 128, 200)
(52, 200)
mixture_devtrain_glas

(127, 128, 200)
(127, 200)
mixture_devtrain_glassbreak_127_f36248b28ce1f017d2f7ddf87afab2bc.wav
854
865
before
823
1023
after
823
1023
(128, 200)
temp_train_labels.shape is: (200,)
(128, 128, 200)
(128, 200)
mixture_devtrain_glassbreak_128_d9ef60b4ceabaf0e395f56d4fef72dd2.wav
69
98
before
37
237
after
37
237
(128, 200)
temp_train_labels.shape is: (200,)
(129, 128, 200)
(129, 200)
mixture_devtrain_glassbreak_129_c36a4a83bbe885a71d1e3a334e4c8fbe.wav
106
155
before
100
300
after
100
300
(128, 200)
temp_train_labels.shape is: (200,)
(130, 128, 200)
(130, 200)
mixture_devtrain_glassbreak_130_5c241831dcbfc79b999098828dfe2da7.wav
1042
1065
before
1002
1202
after
1002
1202
(128, 200)
temp_train_labels.shape is: (200,)
(131, 128, 200)
(131, 200)
mixture_devtrain_glassbreak_131_b904b7e0697cd02a7eb92a403a813f41.wav
691
725
before
666
866
after
666
866
(128, 200)
temp_train_labels.shape is: (200,)
(132, 128, 200)
(132, 200)
mixture_devtrain_glassbreak_132_3bb0ccdaeb1ac19f6f1138c8b2c87ea9.wav
69
98

mixture_devtrain_glassbreak_191_23ed19409c5ce0843f8965e3fe862977.wav
282
341
before
143
343
after
143
343
(128, 200)
temp_train_labels.shape is: (200,)
(192, 128, 200)
(192, 200)
mixture_devtrain_glassbreak_192_22255f1283389cb6847eecf5dda7488b.wav
40
53
before
36
236
after
36
236
(128, 200)
temp_train_labels.shape is: (200,)
(193, 128, 200)
(193, 200)
mixture_devtrain_glassbreak_193_38f8dbf321887fedb474821688540b41.wav
540
595
before
488
688
after
488
688
(128, 200)
temp_train_labels.shape is: (200,)
(194, 128, 200)
(194, 200)
mixture_devtrain_glassbreak_194_1490621b682d582f0872822262ce71e3.wav
130
155
before
48
248
after
48
248
(128, 200)
temp_train_labels.shape is: (200,)
(195, 128, 200)
(195, 200)
mixture_devtrain_glassbreak_195_d20eda7f121e9de6eabd6a107dfb1f24.wav
1028
1078
before
990
1190
after
990
1190
(128, 200)
temp_train_labels.shape is: (200,)
(196, 128, 200)
(196, 200)
mixture_devtrain_glassbreak_196_f9f028f27a5ff4c2024b9cab3aaf6019.wav
123
142
before
79
279
after
79
279
(12

mixture_devtrain_glassbreak_249_68d5570ca79e6ca175126edd310ed3a9.wav
854
865
before
768
968
after
768
968
(128, 200)
temp_train_labels.shape is: (200,)
(250, 128, 200)
(250, 200)
mixture_devtrain_glassbreak_250_b150ebcc04519b6f0243cc0f28c0f0e3.wav
80
163
before
73
273
after
73
273
(128, 200)
temp_train_labels.shape is: (200,)
(251, 128, 200)
(251, 200)
mixture_devtrain_glassbreak_251_4870a4fd799e7880742b012d8814d6fd.wav
996
1031
before
947
1147
after
947
1147
(128, 200)
temp_train_labels.shape is: (200,)
(252, 128, 200)
(252, 200)
mixture_devtrain_glassbreak_252_4112ca9d4b5f6663965b60c950233ddc.wav
1028
1078
before
999
1199
after
999
1199
(128, 200)
temp_train_labels.shape is: (200,)
(253, 128, 200)
(253, 200)
mixture_devtrain_glassbreak_253_56038dbcf540d123c60751db4ab491a7.wav
117
146
before
59
259
after
59
259
(128, 200)
temp_train_labels.shape is: (200,)
(254, 128, 200)
(254, 200)
mixture_devtrain_glassbreak_254_b6b609ed9e4cc85228974538b038621e.wav
969
1079
before
920
1120
after
920

...

(4807, 128, 200)
(4807, 200)
mixture_devtrain_glassbreak_4814_4c95bce9559d85fcb2a3d6a895d41a87.wav
40
142
before
8
208
after
8
208
(128, 200)
temp_train_labels.shape is: (200,)
(4808, 128, 200)
(4808, 200)
mixture_devtrain_glassbreak_4815_bb676ce490604605b21312c54527d24c.wav
393
425
before
268
468
after
268
468
(128, 200)
temp_train_labels.shape is: (200,)
(4809, 128, 200)
(4809, 200)
mixture_devtrain_glassbreak_4816_1a2587af290cf33e9346a7ee583e69bf.wav
880
907
before
808
1008
after
808
1008
(128, 200)
temp_train_labels.shape is: (200,)
(4810, 128, 200)
(4810, 200)
mixture_devtrain_glassbreak_4817_a818d88c2e73bb2a29fd54a304e266f9.wav
831
869
before
742
942
after
742
942
(128, 200)
temp_train_labels.shape is: (200,)
(4811, 128, 200)
(4811, 200)
mixture_devtrain_glassbreak_4818_2f11f0a17d51d2a1ce51be4bbcc236c3.wav
656
701
before
578
778
after
578
778
(128, 200)
temp_train_labels.shape is: (200,)
(4812, 128, 200)
(4812, 200)
mixture_devtrain_glassbreak_4819_2a3eeac428764064c889033ce39c211

142
before
25
225
after
25
225
(128, 200)
temp_train_labels.shape is: (200,)
(4855, 128, 200)
(4855, 200)
mixture_devtrain_glassbreak_4862_3fd71a11f5ab5fd3965af843716edbac.wav
540
595
before
468
668
after
468
668
(128, 200)
temp_train_labels.shape is: (200,)
(4856, 128, 200)
(4856, 200)
mixture_devtrain_glassbreak_4863_243dd1d926a8e5958e6e3ec6bad1f010.wav
436
478
before
378
578
after
378
578
(128, 200)
temp_train_labels.shape is: (200,)
(4857, 128, 200)
(4857, 200)
mixture_devtrain_glassbreak_4864_87f03b85d78e92670967fda6fcab0e2c.wav
553
576
before
385
585
after
385
585
(128, 200)
temp_train_labels.shape is: (200,)
(4858, 128, 200)
(4858, 200)
mixture_devtrain_glassbreak_4865_7daf80fd02e32e9144822998489bca0e.wav
1028
1078
before
917
1117
after
917
1117
(128, 200)
temp_train_labels.shape is: (200,)
(4859, 128, 200)
(4859, 200)
mixture_devtrain_glassbreak_4866_973bc9fd2ff330c41ce68f0b60c5cc79.wav
216
270
before
95
295
after
95
295
(128, 200)
temp_train_labels.shape is: (200,)
(4860, 128,

(4903, 128, 200)
(4903, 200)
mixture_devtrain_glassbreak_4910_73523e723be103d426d05fed6cc9ee19.wav
199
282
before
139
339
after
139
339
(128, 200)
temp_train_labels.shape is: (200,)
(4904, 128, 200)
(4904, 200)
mixture_devtrain_glassbreak_4911_1ba38be9e6d39efc5d823dc6c70ff383.wav
529
622
before
487
687
after
487
687
(128, 200)
temp_train_labels.shape is: (200,)
(4905, 128, 200)
(4905, 200)
mixture_devtrain_glassbreak_4912_d655b5a59555b9daf3f4786042891964.wav
192
316
before
186
386
after
186
386
(128, 200)
temp_train_labels.shape is: (200,)
(4906, 128, 200)
(4906, 200)
mixture_devtrain_glassbreak_4913_93a8c82204b4e4dc468ffd2366da7a11.wav
331
406
before
246
446
after
246
446
(128, 200)
temp_train_labels.shape is: (200,)
(4907, 128, 200)
(4907, 200)
mixture_devtrain_glassbreak_4914_3f6c36ddcf0166b7311aa3ae45cb13f5.wav
1024
1128
before
1022
1222
after
1022
1222
(128, 200)
temp_train_labels.shape is: (200,)
(4908, 128, 200)
(4908, 200)
mixture_devtrain_glassbreak_4915_f1c8f202b85eb288f972b1

(4951, 128, 200)
(4951, 200)
mixture_devtrain_glassbreak_4958_db8cb8d332ec2577e680fa18d786d866.wav
216
270
before
143
343
after
143
343
(128, 200)
temp_train_labels.shape is: (200,)
(4952, 128, 200)
(4952, 200)
mixture_devtrain_glassbreak_4959_123687c0a655b63fd87788cd5da14081.wav
1234
1286
before
1157
1305
after
1105
1305
(128, 200)
temp_train_labels.shape is: (200,)
(4953, 128, 200)
(4953, 200)
mixture_devtrain_glassbreak_4960_99d99d20cbc53d73f60bb28eb3d1dbd3.wav
69
98
before
5
205
after
5
205
(128, 200)
temp_train_labels.shape is: (200,)
(4954, 128, 200)
(4954, 200)
mixture_devtrain_glassbreak_4961_3cf3971e0c420850076ad142b198205e.wav
216
270
before
99
299
after
99
299
(128, 200)
temp_train_labels.shape is: (200,)
(4955, 128, 200)
(4955, 200)
mixture_devtrain_glassbreak_4962_03a5aa677afc5988d747eb27e7744a12.wav
192
316
before
191
391
after
191
391
(128, 200)
temp_train_labels.shape is: (200,)
(4956, 128, 200)
(4956, 200)
mixture_devtrain_glassbreak_4963_559d08ca19c06f99b3995432ecd933

(4999, 128, 200)
(4999, 200)
mixture_devtrain_glassbreak_5006_a004b22835d48296bc22f91fbef774b0.wav
69
145
before
47
247
after
47
247
(128, 200)
temp_train_labels.shape is: (200,)
(5000, 128, 200)
(5000, 200)


In [4]:
train_frames = np.array(d['train_frames'])
train_labels = np.array(d['train_labels'])

In [5]:
f = open('Datasets_legMel_44100_128/train_5000_glassbreak_Prob1_44100_128_framelevel_calssifier.pkl','wb')
pickle.dump(d,f)
f.close()

In [6]:
frames_per_sample = 200

def create_melspec_val(path):
    val_frames=[]
    val_labels=[]
    with open('MyCode/data/TUT-rare-sound-events-2017-development/devtrain_3000_2/2d5cfbf13394fc98749052ea7d64893f/meta/event_list_devtrain_glassbreak.csv', 'rt') as val_file:
        reader = list(csv.reader(val_file, delimiter='\n'))
        
        for row,i in zip(reader,range(1000)):
            fields= str(row).split(',')
            
            temp_val_frames =[]
            temp_val_labels = []
                    
            if(len(fields)>1):
                if(fields[1] and fields[2]):
                    file_name = fields[0].replace("[\'", "").replace("\']", "")
                    melspec = imageio.imread(path + str(i) + '.png')
                    print(file_name)
                    event_start_index = int(np.round(float(fields[1])/30 * 1305))
                    event_finish_index = int(np.round(float(fields[2])/30 * 1305))
                    event_length = event_finish_index - event_start_index 
                    print(event_start_index)
                    print(event_finish_index)
                    
                    #calculating sample start and finish indices, randomly
                    sample_start = randint(max(0,event_start_index - (200-event_length)),event_start_index)
                    sample_finish = min(1305,sample_start+frames_per_sample)
                    
                    print('before')
                    print(sample_start)
                    print(sample_finish)
                    sample_length = sample_finish - sample_start 
                    if sample_length < frames_per_sample:#if we don't have enough negative samples after event
                        sample_start = sample_start - (frames_per_sample - sample_length)
                    
                    print('after')
                    print(sample_start)
                    print(sample_finish)
                    
                    temp1 = np.array(melspec)
                    temp_val_frames = temp1[:,sample_start:sample_finish]
                    
                    print(np.array(temp_val_frames).shape)
                    for ind in range(sample_start,event_start_index):
                        temp_val_labels.append(0)
                    for ind in range(event_start_index,event_finish_index):
                        temp_val_labels.append(1)
                    for ind in range(event_finish_index,sample_finish):
                        temp_val_labels.append(0)
                    
                    print('temp_val_labels.shape is: '+str(np.array(temp_val_labels).shape))   
                    temp2 = np.array(temp_val_frames)
                    temp2 = temp2.reshape(128,200)
                    val_frames.append(temp2)
                    
                    temp3 = np.array(temp_val_labels)
                    temp3 = temp3.reshape(200)
                    val_labels.append(temp3)
                    
                    print(np.array(val_frames).shape)
                    print(np.array(val_labels).shape)
            if(i%500==0):
                print('Reading files: '+str(i))
    
    val_file.close()
    val_data = {'val_frames':val_frames,
                 'val_labels':val_labels}
    return val_data

In [7]:
val_file_path = 'Datasets_legMel_44100_128/val_1000_glassbreak_44100_128/'
v={}
v = create_melspec_val(val_file_path)

mixture_devtrain_glassbreak_000_3ab1602ca04576d1bb37b79a153c0d6c.wav
958
1052
before
956
1156
after
956
1156
(128, 200)
temp_val_labels.shape is: (200,)
(1, 128, 200)
(1, 200)
Reading files: 0
mixture_devtrain_glassbreak_005_d2327e3188b21ccb809a694029a1def0.wav
382
427
before
227
427
after
227
427
(128, 200)
temp_val_labels.shape is: (200,)
(2, 128, 200)
(2, 200)
mixture_devtrain_glassbreak_006_d03b3957f5b0547b333f2b4b174f1168.wav
101
146
before
97
297
after
97
297
(128, 200)
temp_val_labels.shape is: (200,)
(3, 128, 200)
(3, 200)
mixture_devtrain_glassbreak_013_b1c6b0e6b53a8fa51192266b4d2d99cd.wav
1166
1241
before
1142
1305
after
1105
1305
(128, 200)
temp_val_labels.shape is: (200,)
(4, 128, 200)
(4, 200)
mixture_devtrain_glassbreak_015_e45580c9d87d3b9f84d740a3a161514d.wav
1210
1258
before
1082
1282
after
1082
1282
(128, 200)
temp_val_labels.shape is: (200,)
(5, 128, 200)
(5, 200)
mixture_devtrain_glassbreak_016_912a8624fce38b450be6d66b62d3b450.wav
430
512
before
385
585
after
385
585

mixture_devtrain_glassbreak_134_a5b66d6598aec932a59519b7c9121a39.wav
442
572
before
429
629
after
429
629
(128, 200)
temp_val_labels.shape is: (200,)
(60, 128, 200)
(60, 200)
mixture_devtrain_glassbreak_138_29e28892ba6aac6913bce3b156e7f036.wav
1138
1176
before
1052
1252
after
1052
1252
(128, 200)
temp_val_labels.shape is: (200,)
(61, 128, 200)
(61, 200)
mixture_devtrain_glassbreak_141_b853beead3943c4fb121b9db9a2b7add.wav
1159
1170
before
1037
1237
after
1037
1237
(128, 200)
temp_val_labels.shape is: (200,)
(62, 128, 200)
(62, 200)
mixture_devtrain_glassbreak_145_0a6c09a98f09d92efa5b9a6d7ca576a6.wav
165
226
before
161
361
after
161
361
(128, 200)
temp_val_labels.shape is: (200,)
(63, 128, 200)
(63, 200)
mixture_devtrain_glassbreak_147_a84c13b6feb573aadb08a165226db3f3.wav
909
942
before
783
983
after
783
983
(128, 200)
temp_val_labels.shape is: (200,)
(64, 128, 200)
(64, 200)
mixture_devtrain_glassbreak_148_a5433d51c17a92c72f883bd4fd9b3f36.wav
392
432
before
334
534
after
334
534
(128, 2

37
237
(128, 200)
temp_val_labels.shape is: (200,)
(107, 128, 200)
(107, 200)
mixture_devtrain_glassbreak_215_1c2afbbcd648ae8f46889be01ba17912.wav
211
271
before
79
279
after
79
279
(128, 200)
temp_val_labels.shape is: (200,)
(108, 128, 200)
(108, 200)
mixture_devtrain_glassbreak_218_01a259d78f2d4ddb38bcb9f5142dd77b.wav
659
703
before
634
834
after
634
834
(128, 200)
temp_val_labels.shape is: (200,)
(109, 128, 200)
(109, 200)
mixture_devtrain_glassbreak_221_5c7642bc4e01b1d4140fed7d44373d83.wav
215
245
before
163
363
after
163
363
(128, 200)
temp_val_labels.shape is: (200,)
(110, 128, 200)
(110, 200)
mixture_devtrain_glassbreak_223_8727b2d07a27482219578de167c3be60.wav
1210
1258
before
1164
1305
after
1105
1305
(128, 200)
temp_val_labels.shape is: (200,)
(111, 128, 200)
(111, 200)
mixture_devtrain_glassbreak_224_a6a522c51f83e1f7b79bdaa4958dca96.wav
754
815
before
661
861
after
661
861
(128, 200)
temp_val_labels.shape is: (200,)
(112, 128, 200)
(112, 200)
mixture_devtrain_glassbreak_227_4

mixture_devtrain_glassbreak_375_180c11227bc2ad306946afebc8772173.wav
695
730
before
542
742
after
542
742
(128, 200)
temp_val_labels.shape is: (200,)
(192, 128, 200)
(192, 200)
mixture_devtrain_glassbreak_376_1aaca744a9e2ce3ed61f3b26f8db22fd.wav
611
648
before
591
791
after
591
791
(128, 200)
temp_val_labels.shape is: (200,)
(193, 128, 200)
(193, 200)
mixture_devtrain_glassbreak_380_9e7b2afd8e040765b1e9c347efd48a26.wav
721
849
before
676
876
after
676
876
(128, 200)
temp_val_labels.shape is: (200,)
(194, 128, 200)
(194, 200)
mixture_devtrain_glassbreak_381_bf4254c53a383dc44a42de9e83680f81.wav
703
722
before
644
844
after
644
844
(128, 200)
temp_val_labels.shape is: (200,)
(195, 128, 200)
(195, 200)
mixture_devtrain_glassbreak_383_178f4a12f76daced9844440cab712982.wav
993
1023
before
957
1157
after
957
1157
(128, 200)
temp_val_labels.shape is: (200,)
(196, 128, 200)
(196, 200)
mixture_devtrain_glassbreak_387_fac15e6595aff87093020bb5ab15e6aa.wav
165
226
before
89
289
after
89
289
(128, 20

(128, 200)
temp_val_labels.shape is: (200,)
(264, 128, 200)
(264, 200)
mixture_devtrain_glassbreak_523_98a9829c3df2c04fac7c067ea77608d1.wav
1213
1271
before
1089
1289
after
1089
1289
(128, 200)
temp_val_labels.shape is: (200,)
(265, 128, 200)
(265, 200)
mixture_devtrain_glassbreak_524_1a08b45cbb6a4056d93fa46479ba49b1.wav
538
562
before
453
653
after
453
653
(128, 200)
temp_val_labels.shape is: (200,)
(266, 128, 200)
(266, 200)
mixture_devtrain_glassbreak_526_1e5b2b5978bc50c6726bcb7d60c232a2.wav
659
703
before
647
847
after
647
847
(128, 200)
temp_val_labels.shape is: (200,)
(267, 128, 200)
(267, 200)
mixture_devtrain_glassbreak_528_0bb31e726ab79fe675f79ebcecbcfd5c.wav
938
982
before
900
1100
after
900
1100
(128, 200)
temp_val_labels.shape is: (200,)
(268, 128, 200)
(268, 200)
mixture_devtrain_glassbreak_531_141387a3b8b97e9b5e75f53ae4df8b1a.wav
292
385
before
288
488
after
288
488
(128, 200)
temp_val_labels.shape is: (200,)
(269, 128, 200)
(269, 200)
mixture_devtrain_glassbreak_533_c145

(328, 200)
mixture_devtrain_glassbreak_647_ab85d2a076d303fe69b6e141f2b8dbbc.wav
334
409
before
285
485
after
285
485
(128, 200)
temp_val_labels.shape is: (200,)
(329, 128, 200)
(329, 200)
mixture_devtrain_glassbreak_649_fe013eec28f5bf728b680d6754502434.wav
752
786
before
714
914
after
714
914
(128, 200)
temp_val_labels.shape is: (200,)
(330, 128, 200)
(330, 200)
mixture_devtrain_glassbreak_650_a2677a25ff580653fcd88be3271e3804.wav
32
115
before
25
225
after
25
225
(128, 200)
temp_val_labels.shape is: (200,)
(331, 128, 200)
(331, 200)
mixture_devtrain_glassbreak_651_e1ae2d523a884d4603b69433775272b7.wav
695
730
before
573
773
after
573
773
(128, 200)
temp_val_labels.shape is: (200,)
(332, 128, 200)
(332, 200)
mixture_devtrain_glassbreak_652_116f3664a4f76ce866284e0c2385cff3.wav
611
648
before
491
691
after
491
691
(128, 200)
temp_val_labels.shape is: (200,)
(333, 128, 200)
(333, 200)
mixture_devtrain_glassbreak_653_e8a5196a95596c135b6060ab969a7c03.wav
322
358
before
158
358
after
158
358
(

(384, 128, 200)
(384, 200)
mixture_devtrain_glassbreak_767_a9ac409632256447c0068dd77d385f60.wav
1138
1176
before
977
1177
after
977
1177
(128, 200)
temp_val_labels.shape is: (200,)
(385, 128, 200)
(385, 200)
mixture_devtrain_glassbreak_769_c76d23e5594fae24f016e0fb57136912.wav
659
703
before
649
849
after
649
849
(128, 200)
temp_val_labels.shape is: (200,)
(386, 128, 200)
(386, 200)
mixture_devtrain_glassbreak_770_1a409e91bd26e396eb63489bf60bb3cf.wav
695
730
before
672
872
after
672
872
(128, 200)
temp_val_labels.shape is: (200,)
(387, 128, 200)
(387, 200)
mixture_devtrain_glassbreak_772_5f0fb353fd1d0560eb7b70d90f333bb1.wav
649
708
before
588
788
after
588
788
(128, 200)
temp_val_labels.shape is: (200,)
(388, 128, 200)
(388, 200)
mixture_devtrain_glassbreak_774_488bd0c779dc4fc991ea5f3fca1c0eba.wav
182
211
before
73
273
after
73
273
(128, 200)
temp_val_labels.shape is: (200,)
(389, 128, 200)
(389, 200)
mixture_devtrain_glassbreak_775_437607e556ffd0aa32cc1ed998a46a6c.wav
721
849
before
69

mixture_devtrain_glassbreak_859_4181f567cc9781a51bb7911908f20537.wav
686
718
before
578
778
after
578
778
(128, 200)
temp_val_labels.shape is: (200,)
(437, 128, 200)
(437, 200)
mixture_devtrain_glassbreak_864_70f30fc10cbfd84c105664b8e035a82f.wav
1060
1124
before
1054
1254
after
1054
1254
(128, 200)
temp_val_labels.shape is: (200,)
(438, 128, 200)
(438, 200)
mixture_devtrain_glassbreak_865_d2c21233cd75dba0e04e885952a25a24.wav
878
909
before
716
916
after
716
916
(128, 200)
temp_val_labels.shape is: (200,)
(439, 128, 200)
(439, 200)
mixture_devtrain_glassbreak_866_d0822f927ae6d08e7eef1144e479deb3.wav
226
304
before
106
306
after
106
306
(128, 200)
temp_val_labels.shape is: (200,)
(440, 128, 200)
(440, 200)
mixture_devtrain_glassbreak_867_80cd832364c7c39864c031ecbe85ee73.wav
163
236
before
109
309
after
109
309
(128, 200)
temp_val_labels.shape is: (200,)
(441, 128, 200)
(441, 200)
mixture_devtrain_glassbreak_868_7565d2b756869a8731c13915daa45c95.wav
865
935
before
828
1028
after
828
1028
(

mixture_devtrain_glassbreak_949_7241f3a12e244f24e4d77b17d4f90e02.wav
1213
1271
before
1188
1305
after
1105
1305
(128, 200)
temp_val_labels.shape is: (200,)
(487, 128, 200)
(487, 200)
mixture_devtrain_glassbreak_950_39d4def518df5dff41f983ec4f41e128.wav
395
456
before
265
465
after
265
465
(128, 200)
temp_val_labels.shape is: (200,)
(488, 128, 200)
(488, 200)
mixture_devtrain_glassbreak_952_55aa5ba894e57ef5c458f4fa1ab5a20f.wav
190
269
before
164
364
after
164
364
(128, 200)
temp_val_labels.shape is: (200,)
(489, 128, 200)
(489, 200)
mixture_devtrain_glassbreak_953_5d7818fd095d9089c6a8c0471fa47331.wav
1081
1126
before
935
1135
after
935
1135
(128, 200)
temp_val_labels.shape is: (200,)
(490, 128, 200)
(490, 200)
mixture_devtrain_glassbreak_957_27e25eccb7a38b779528d2de415c0646.wav
584
597
before
569
769
after
569
769
(128, 200)
temp_val_labels.shape is: (200,)
(491, 128, 200)
(491, 200)
mixture_devtrain_glassbreak_959_a98a2209ed24794cdbdb53bd23b6b37f.wav
608
708
before
558
758
after
558
758

In [8]:
val_frames = np.array(v['val_frames'])[0:500]
val_labels = np.array(v['val_labels'])[0:5]

In [9]:
f = open('Datasets_legMel_44100_128/val_1000_glassbreak_Prob1_44100_128__framelevel_calssifier.pkl','wb')
pickle.dump(v,f)
f.close()

In [10]:
def get_model():
    
    
    inp = Input(shape=(128,200,1))
    x = Conv2D(16, (5,5), activation='relu', padding="valid")(inp)
    x = MaxPooling2D((5,5))(x)
    x = Dropout(rate=0.3)(x)
    
    
    
    x = Conv2D(64, (3,3), activation='relu', padding="valid")(x)
    x = MaxPooling2D((3,3))(x)
    x = Dropout(rate=0.3)(x)
    
    x = Conv2D(128, (3,3), activation='relu', padding="valid")(x)
    x = MaxPooling2D((3,3))(x)
    x = Dropout(rate=0.3)(x)
    
    x = Flatten()(x)
    
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.3)(x)
    out = Dense(200, activation='sigmoid')(x)
    
    
    
    model = Model(inputs=inp, outputs=out)
    opt = optimizers.Adam(lr=0.001)

    model.compile(optimizer=opt, loss=losses.binary_crossentropy, metrics=['acc'])
    return model


In [11]:
my_model = get_model()

In [12]:
my_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 200, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 124, 196, 16)      416       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 39, 16)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 24, 39, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 22, 37, 64)        9280      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 12, 64)         0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 7, 12, 64)         0         
__________

In [13]:
earlystop = EarlyStopping(monitor='val_acc', min_delta=0.0001, patience=20, mode='auto')

In [14]:
train_frames = np.array(train_frames).reshape(5000,128,200,1)
val_frames = np.array(val_frames).reshape(500,128,200,1)
train_labels = np.array(train_labels).reshape(5000,200)
val_labels = np.array(val_labels).reshape(500,200)

ValueError: cannot reshape array of size 12928000 into shape (500,128,200,1)

In [181]:
my_model.fit(x=train_frames,y=train_labels,batch_size=200,epochs=150, 
             validation_data=(val_frames, val_labels), callbacks=[earlystop])

Train on 5000 samples, validate on 500 samples
Epoch 1/150
5000/5000 [==============================] - 53s 11ms/step - loss: 1.4878 - acc: 0.5745 - val_loss: 0.6890 - val_acc: 0.6793
Epoch 2/150
5000/5000 [==============================] - 51s 10ms/step - loss: 0.6871 - acc: 0.6748 - val_loss: 0.6846 - val_acc: 0.6923
Epoch 3/150
5000/5000 [==============================] - 51s 10ms/step - loss: 0.6827 - acc: 0.6819 - val_loss: 0.6804 - val_acc: 0.6933
Epoch 4/150
5000/5000 [==============================] - 51s 10ms/step - loss: 0.6785 - acc: 0.6807 - val_loss: 0.6763 - val_acc: 0.6937
Epoch 5/150
5000/5000 [==============================] - 51s 10ms/step - loss: 0.6745 - acc: 0.6761 - val_loss: 0.6723 - val_acc: 0.6935
Epoch 6/150
5000/5000 [==============================] - 50s 10ms/step - loss: 0.6699 - acc: 0.6669 - val_loss: 0.6683 - val_acc: 0.6937
Epoch 7/150
5000/5000 [==============================] - 50s 10ms/step - loss: 0.6617 - acc: 0.6468 - val_loss: 0.6651 - val_acc: 0

KeyboardInterrupt: 